---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [6]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
#df.head()

### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [7]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 


# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [1]:
def answer_one():
    return df['Gold'].idxmax()

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [216]:
def answer_two():
    df["Gold_diff"] = df['Gold'] - df['Gold.1']
    df["Gold_diff"].abs()
    return df["Gold_diff"].idxmax()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [275]:
def answer_three():
    gold_diff = abs(df['Gold'] - df['Gold.1'])
    gold_sum  =  df['Gold'] + df['Gold.1']
    gold_div   = gold_diff.divide(gold_sum)
    df['rel_gold_diff'] = gold_div [(df['Gold'] > 0) & (df['Gold.1'] > 0)]
    return df['rel_gold_diff'].idxmax() 
answer_three()


'Bulgaria'


Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [218]:
def answer_four():
    df['Points'] = df['Gold.2']*3 + df['Silver.2']*2 +df['Bronze.2']*1 
    return df.loc[:,'Points']

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [250]:
census_df = pd.read_csv('census.csv')
station_name = census_df['STNAME'][census_df['SUMLEV']==40]

#for index in value station_name.items():
 #       print(index)

#census_df.set_index(['STNAME','CTYNAME'])


In [2]:
def answer_five():
    count = census_df['STNAME'].value_counts()
    count.idxmax()
    return count.idxmax()

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [252]:
def answer_six():
    state_name = census_df['STNAME'][(census_df['SUMLEV'] ==40)]
    sumdf = pd.DataFrame(columns = ['state','population'])
    for stname in state_name:
        statename_df = census_df[(census_df['STNAME'] == stname ) & (census_df['SUMLEV'] ==50) ]
        statename_df.sort_values('CENSUS2010POP',ascending=False, inplace=True)
        sum3 = statename_df.iloc[0:3,8].sum() 
        sumdf = sumdf.append({'state' : stname , 'population' : sum3 } , ignore_index=True)

    sumdf.sort_values('population',ascending=False, inplace=True)
    return list(sumdf['state'][0:3])

answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [231]:

def answer_seven():
    import pandas as pd
    census_df = pd.read_csv('census.csv')
    selected_columns = ['STNAME',
                        'CTYNAME',
                        'POPESTIMATE2010',
                        'POPESTIMATE2011',
                        'POPESTIMATE2012',
                        'POPESTIMATE2013',
                        'POPESTIMATE2014',
                        'POPESTIMATE2015']
    census_new_df = census_df[selected_columns][census_df['SUMLEV']==50]

    max_ = census_new_df[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis=1)
    min_ = census_new_df[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
    census_new_df['abs_change'] = abs(max_-min_)
    census_new_df.sort_values('abs_change',ascending=False, inplace=True)

    return list(census_new_df['CTYNAME'][0:1])[0]

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [222]:
def answer_eight():
    import pandas as pd
    census_df = pd.read_csv('census.csv')
    
    selected_col = ['STNAME','CTYNAME']
    census_new1_df =  census_df[selected_col][((census_df['REGION'] == 1) |  (census_df['REGION'] == 2)) & (census_df['CTYNAME'].str.match('Washington') )  & (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014'] )   ]
    return  census_new1_df

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
